In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
import torch.optim as optimizers
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.transforms as transforms

np.random.seed(123)
torch.manual_seed(123)

In [2]:
# データの準備
path = "D:Statistics/data/deep_leraning/"
root = os.path.join(path, "torch", "mnist")
transform = transforms.Compose([transforms.ToTensor(), lambda x: x.view(-1)])
mnist_train = datasets.MNIST(root=root, download=True, train=True, transform=transform)
mnist_test = datasets.MNIST(root=root, download=True, train=False, transform=transform)

train_dataloader = DataLoader(mnist_train, batch_size=100, shuffle=True)
test_dataloader = DataLoader(mnist_test, batch_size=100, shuffle=False)

In [9]:
# DNNを定義
class DNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, rate):
        super().__init__()
        self.l1 = nn.Linear(input_dim, hidden_dim)
        self.a1 = nn.ReLU()
        self.d1 = nn.Dropout(rate)
        self.l2 = nn.Linear(hidden_dim, hidden_dim)
        self.a2 = nn.ReLU()
        self.d2 = nn.Dropout(rate)
        self.l3 = nn.Linear(hidden_dim, hidden_dim)
        self.a3 = nn.ReLU()
        self.d3 = nn.Dropout(rate)
        self.l4 = nn.Linear(hidden_dim, output_dim)
        self.layers = [self.l1, self.a1, self.d1, self.l2, self.a2, self.d2, self.l3, self.a3, self.d3, self.l4]

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

In [13]:
# モデルの学習
# アルゴリズムの定義
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
classes = np.unique(train_dataloader.dataset.classes).shape[0]
m = np.prod(train_dataloader.dataset.data.shape[1:])
rate = 0.5
model = DNN(m, 200, classes, rate).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optimizers.SGD(model.parameters(), lr=0.025)

def compute_loss(t, y):
    return criterion(y, t)

def train_step(x, t):
    model.train()
    preds = model(x)
    loss = compute_loss(t, preds)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss, preds

# アルゴリズムの設定
epochs = 50

In [14]:
# 確率的勾配法でモデルを学習
for epoch in range(epochs):
    train_loss = 0.0
    train_acc = 0.0

    for (x, t) in train_dataloader:
        x, t = x.to(device), t.to(device)
        loss, preds = train_step(x, t)
        train_loss += loss.item()
        train_acc += accuracy_score(t.tolist(), preds.argmax(dim=-1).tolist())

    train_loss /= len(train_dataloader)
    train_acc /= len(train_dataloader)
    print("epoch: {}, loss: {:.3}, acc: {:,.3f}".format(epoch+1, train_loss, train_acc))

epoch: 1, loss: 2.13, acc: 0.262
epoch: 2, loss: 1.06, acc: 0.649
epoch: 3, loss: 0.628, acc: 0.807
epoch: 4, loss: 0.493, acc: 0.855
epoch: 5, loss: 0.414, acc: 0.881
epoch: 6, loss: 0.365, acc: 0.895
epoch: 7, loss: 0.32, acc: 0.908
epoch: 8, loss: 0.296, acc: 0.916
epoch: 9, loss: 0.272, acc: 0.923
epoch: 10, loss: 0.254, acc: 0.929
epoch: 11, loss: 0.239, acc: 0.933
epoch: 12, loss: 0.224, acc: 0.937
epoch: 13, loss: 0.213, acc: 0.939
epoch: 14, loss: 0.204, acc: 0.942
epoch: 15, loss: 0.197, acc: 0.945
epoch: 16, loss: 0.191, acc: 0.946
epoch: 17, loss: 0.183, acc: 0.949
epoch: 18, loss: 0.176, acc: 0.950
epoch: 19, loss: 0.166, acc: 0.952
epoch: 20, loss: 0.16, acc: 0.955
epoch: 21, loss: 0.161, acc: 0.954
epoch: 22, loss: 0.155, acc: 0.956
epoch: 23, loss: 0.153, acc: 0.956
epoch: 24, loss: 0.148, acc: 0.958
epoch: 25, loss: 0.143, acc: 0.959
epoch: 26, loss: 0.139, acc: 0.960
epoch: 27, loss: 0.135, acc: 0.961
epoch: 28, loss: 0.132, acc: 0.962
epoch: 29, loss: 0.131, acc: 0.96

In [15]:
# モデルの評価
def test_step(x, t):
    model.eval()
    preds = model(x)
    loss = criterion(preds, t)
    return loss, preds

test_loss = 0.0
test_acc = 0.0

for (x, t) in test_dataloader:
    x, t = x.to(device), t.to(device)
    loss, preds = test_step(x, t)
    test_loss += loss.item()
    test_acc += accuracy_score(t.tolist(), preds.argmax(dim=-1).tolist())

test_loss /= len(test_dataloader)
test_acc /= len(test_dataloader)
print('test_loss: {:.3f}, test_acc: {:.3f}'.format(test_loss, test_acc))

test_loss: 0.081, test_acc: 0.977
